In [1]:
import openai
from qdrant_client import QdrantClient
from qdrant_client.http import models
from langchain.schema import Document

import os
os.chdir('..') 

In [2]:
from src.api.workflows.FAQs.extract_FAQs import obtener_faqs, obtener_faqs_unir
faqs = obtener_faqs(); faqs_unir = obtener_faqs_unir()

In [3]:
openai.api_key = 'sk-proj-y8IHH8cMe6HhmxBmiUvkT3BlbkFJiJ1iiNep9iwvIzYelqCS'

def upload_vectors(collection_name, vectores_array, documents):
    qdrant_client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(size=len(vectores_array[0]), distance=models.Distance.COSINE),
    )
    points = []
    for idx, (vector, chunk) in enumerate(zip(vectores_array, documents)):
        point = models.PointStruct(
            id=idx,
            vector=vector.tolist(),
            payload={"text": chunk.page_content}
          )
        points.append(point)
    qdrant_client.upsert(
        collection_name=collection_name,
        points=points
    )

def encode_text(text):
    response = openai.Embedding.create(
        model="text-embedding-3-large",
        input=text
    )
    return response['data'][0]['embedding']

In [4]:
texts = faqs.split('<h2>')[1:]+faqs_unir.split('<h2>')[1:]
qdrant_client = QdrantClient(
    url="https://b8957c2b-a76c-464e-a03e-fc65ab126491.us-east4-0.gcp.cloud.qdrant.io", 
    api_key="UJiDIyKcqEvrew0gcVY8PxhjwXEyEPiPjonv-Ab_jeaE7ir3-_p7DA",
)

import numpy
vectores = [encode_text(text) for text in texts]
documents = [Document(page_content=text) for text in texts]
upload_vectors('MoodleIA', numpy.array(vectores), documents)
print("Vectores Len:", len(vectores))

C:\Users\juanmanuel.castillo\AppData\Local\Temp\ipykernel_7136\237311899.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


Vectores Len: 31


In [6]:
query='Buenas tardes don Antonio soy Lugdy Ropero dé Ocaña norte de Santander Colombia. Lo que pasa es que desde que empezaron los cursos he querido realizarlos pero no me he podido registrar en la plataforma.'
search_result = qdrant_client.search(
    collection_name='MoodleIA',
    query_vector=encode_text(query),
    limit=3
)
search_result

[ScoredPoint(id=13, version=0, score=0.49392346, payload={'text': '13. Cuando ingreso en la plataforma, no aparece mi curso</h2><div><p>Asegúrate de que bajas en la página de inicio hasta la sección "Mis cursos". Es en esa sección donde se encuentra el acceso. Si no, dilo a tu tutor. <em>Lo primero que deberá hacer el tutor es acceder con tu nombre de usuario y contraseña para comprobar que tu problema es real: suele ocurrir que se trata de un despiste.</em></p><p>Ejemplo típico: una persona acaba de escribir diciendo que no tiene acceso al curso sobre "Educación afectivo-sexual". Accedo con sus datos de usuario y veo esto (sí que tiene acceso):</p><br/><p class="centrar"><img alt="Mis cursos" class="responsive" src="https://becat.online/FAQ/img/miscursos.png"/> </p></div>'}, vector=None, shard_key=None),
 ScoredPoint(id=1, version=0, score=0.47508082, payload={'text': '1. No puedo acceder con mis datos de usuario</h2><div><p>Asegúrate en primer lugar que introduces bien los datos:</p>